In [6]:
from jupyter_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
#import dash_table
from dash.dependencies import Input, Output

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
import base64
# importing python module for accessing data
from moduleFour import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "test"
password = "test"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))





#########################
# Dashboard Layout / View
#########################

app = JupyterDash('SimpleExample')


#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Justin Farquhar Project 2 Dashboard'))),
    html.Hr(),
    html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image'),
    html.Div(
        dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
            {'label': 'Disaster Rescue and Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset'
        )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable='multi',
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10  
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id', "data"),
              Output('datatable-id', "columns"),
             [Input('filter-type', 'value')
             ])
        
def update_dashboard(filter_type):
    df = pd.DataFrame.from_records(shelter.read({}))
    if filter_type:
            # Filter for water rescue
        if filter_type == 'water':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {
                    "$in": [
                        "Labrador Retriever Mix",
                        "Chesapeake Bay Retriever",
                        "Newfoundland"
                    ]
                },
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks":
                {"$gte":26.0, "$lte":156.0} 
            }))
        # Filter for mountain or wilderness rescue
        elif filter_type == 'mount':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["German Shepherd","Alaskan Malamute","Old English Sheepdog", "Siberian Husky",
                              "Rottweiler"
                             ]},
                "sex_upon_outcome": "Intact Male",
                "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0} 
            }))
        # Filter for disaster rescue or individual tracking
        elif filter_type == 'disaster':
            df = pd.DataFrame.from_records(shelter.read({
                "animal_type": "Dog",
                "breed": {"$in": ["Doberman Pinscher","German Shepherd","Golden Retriever",
                              "Bloodhound", "Rottweiler"
                             ]},
                "sex_upon_outcome": "Intact Female",
                "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0} 
            }))

    if df.empty:
        df = pd.DataFrame(columns=["No records found for the selected filter"])
    columns = [{"name": i, "id": i} for i in df.columns]   
    return df.to_dict('records'), columns

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Chart of choice
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")
    ])
def update_graph(viewData):
    if viewData is None or len(viewData) == 0:
        return [html.Div("No data available for visualization")]

    dff = pd.DataFrame.from_dict(viewData)

    # Group by breed and count occurrences
    grouped_df = dff.groupby('breed').size().reset_index(name='count')

    fig = px.pie(grouped_df, values='count', names='breed', title='Available Dogs by Breed')

    return [
        dcc.Graph(
            figure=fig
        )
    ]
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id', "selected_rows")])

def update_map(viewData, selected_rows):
    dff = pd.DataFrame.from_dict(viewData)
    
    markers = []
    if selected_rows:
        for row in selected_rows:
            lat = dff['location_lat'][row]
            lon = dff['location_long'][row]
            markers.append(
                dl.Marker(position=[lat, lon],
                         children=[
                             dl.Tooltip(dff['breed'][row]),
                             dl.Popup([
                                 html.H1("Animal Name"),
                                 html.P(dff['name'][row])
                             ])
                         ]))
    default_map = dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
        dl.TileLayer(id="base-layer-id")
    ])
    
    return [default_map] if not markers else [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
      dl.TileLayer(id="base-layer-id"),
      *markers
    ])]
      
app.run_server(mode='inline', debug=True)